# Shark attacks data cleaning

- Importación de las librerías necesarias

In [1]:
import pandas as pd
import numpy as np
import re
import Src.SharkFn as Fn

- Creación del data frame Madre

In [2]:
df_Madre = pd.read_csv('Input/GSAF5.csv', encoding = "ISO-8859-1")

In [3]:
df_Madre.columns.values

array(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area',
       'Location', 'Activity', 'Name', 'Sex ', 'Age', 'Injury',
       'Fatal (Y/N)', 'Time', 'Species ', 'Investigator or Source', 'pdf',
       'href formula', 'href', 'Case Number.1', 'Case Number.2',
       'original order', 'Unnamed: 22', 'Unnamed: 23'], dtype=object)

In [4]:
print (df_Madre.shape)

(5992, 24)


In [5]:
null_cols = df_Madre.isnull().sum()
null_cols
null_cols[null_cols > 0]

Country                     43
Area                       402
Location                   496
Activity                   527
Name                       200
Sex                        567
Age                       2681
Injury                      27
Fatal (Y/N)                 19
Time                      3213
Species                   2934
Investigator or Source      15
href formula                 1
href                         3
Unnamed: 22               5991
Unnamed: 23               5990
dtype: int64

In [6]:
# set(df_Madre['Activity']) # **!**
# set(df_Madre['Time']) # **!**
set(df_Madre['Species ']) # **!**
# set(df_Madre['Country']) # **!**

# print(df_Madre['Type'].value_counts())
# print(df_Madre['Sex '].value_counts())
# print(df_Madre['Fatal (Y/N)'].value_counts())

# set(df_Madre['Date'])
# set(df_Madre['Year'])

# set(df_Madre['Area'])
# set(df_Madre['Location'])

# set(df_Madre['Name'])
# set(df_Madre['Age'])
# set(df_Madre['Injury'])

    #-----------#
#          set(df_Madre['Case Number'])
        
        # set(df_Madre['Investigator or Source'])
        # set(df_Madre['pdf'])
        # set(df_Madre['href formula'])
        # set(df_Madre['href'])
        # set(df_Madre['Case Number.1'])
        # set(df_Madre['Case Number.2'])
        # set(df_Madre['original order'])
        # set(df_Madre['Unnamed: 22'])
        # set(df_Madre['Unnamed: 23'])

{nan,
 "Tiger shark, 2.4 m to 2.7 m [8' to 9'] ",
 'Raggedtooth shark',
 "Tiger shark, 3 m to 4.9 m [10' to 16'] ",
 "5' shark",
 'Grey reef shark, Identified as C. menisorrah, by E.S. Hobson, F. Mautin & E.S. Reese (1961)',
 "Tiger shark, 1.8 m [6']",
 "White shark, 13'",
 'Bull or tiger shark',
 '20 kg shark',
 "4.9 m [16'] shark",
 "White shark, 5' to 6' juvenile ",
 '1.8 to 2 m C. albimarginatus',
 "Tiger shark, 4.3 m [14'], 3 tooth fragments retrieved from his wounds",
 "1.8 m [6'] shark ",
 "Blue pointer, 11' ",
 '1.7 m shark',
 "Lemon shark, 9' ",
 "Raggedtooth shark, 2.5 m [8.25'] ",
 "3.2 m [10.5'] shark",
 'Tiger shark,  3.6 m [11\'9"], 850-kg [1874-lb] female, contained Girvan\'s remains ',
 'White shark, 4.5m',
 "Mako shark, 1.8 m to 2.1 m [6' to 7']  with hook & wire leader caught in mouth",
 '2 m to  3 m shark',
 '2 m cow shark',
 "2.4 m [8'] white shark, species identity confirmed by witnesses & tooth pattern in surfboard",
 'C. leucas tooth fragment recovered from kayak

In [7]:
to_drop = ['Investigator or Source','pdf','href formula','href',
           'Case Number.1','Case Number.2','original order','Unnamed: 22','Unnamed: 23']
df_raw = df_Madre
df_raw.drop(to_drop, inplace=True, axis=1)


In [8]:
df_raw.rename(columns = {'Species ': 'Species',
                        'Sex ': 'Sex',
                        'Fatal (Y/N)' : 'Fatal'}, inplace=True)
df_raw.columns.values

array(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area',
       'Location', 'Activity', 'Name', 'Sex', 'Age', 'Injury', 'Fatal',
       'Time', 'Species'], dtype=object)

In [9]:
def clean_last_ws (serie):
    return serie.replace("\s$", "")

def clean_first_ws (serie):
    return serie.replace("^\s", "")

def clean_mult_ws (serie):
    return serie.replace("\s+", " ")

def clean_weird_ch (serie):
    return serie.replace("[\[\]\,\"\'\\\.\-\?\<\>\*\&\;\+\(\)]", "")

def clean_num (serie):
    return serie.replace("[\d]", "")

def clean_1to2_ch (serie):
    aux = serie.replace("\s\w{1,2}\s", " ")
    aux1 = aux.replace("\s\w{1,2}\s", " ")
    aux2 = aux1.replace("\s\w{1,2}\s", " ")
    return aux2.replace("\s\w{1,2}\s", " ")
    
def shark_sp (serie):
    return serie.replace(r'(?!\w+\sshark)\b([\S\s]+?)(\b|$)', lambda x: (x.end() - x.start())*'')
    

Limpiando Sex

In [10]:
df_raw.Sex =(clean_last_ws(df_raw.Sex.str))
# df_raw.Sex =(Fn.clean_last_ws(df_raw.Sex.str))

df_raw.loc[(df_raw.Sex == '.')|
           (df_raw.Sex == 'N')|
           (df_raw.Sex == 'lli'), 'Sex'] = np.nan

df_raw.Sex.value_counts(dropna=False)


M      4837
F       585
NaN     570
Name: Sex, dtype: int64

Limpiando Fatal

In [11]:
df_raw.Fatal = clean_first_ws(df_raw.Fatal.str)
df_raw.Fatal = clean_last_ws(df_raw.Fatal.str)
df_raw.Fatal = df_raw.Fatal.str.capitalize()
df_raw.loc[(df_raw.Fatal == '#value!')|
           (df_raw.Fatal == 'F')|
           (df_raw.Fatal =='Unknown'), 'Fatal'] = np.nan

df_raw.Fatal.value_counts(dropna=False)

N      4325
Y      1552
NaN     115
Name: Fatal, dtype: int64

Limpiando Species

In [12]:
df_raw.Species = clean_weird_ch(df_raw.Species.str)
df_raw.Species = clean_1to2_ch(df_raw.Species.str)
df_raw.Species = clean_num(df_raw.Species.str)
df_raw.Species = clean_mult_ws(df_raw.Species.str)
df_raw.Species = clean_last_ws(df_raw.Species.str)
df_raw.Species = clean_first_ws(df_raw.Species.str)


set(df_raw['Species'])

{'',
 'A bluenosed shark',
 'A brown shark',
 'A bull shark according some the divers the boat',
 'A hammerhead shark then to other sharks were said be involved',
 'A large hammerhead shark',
 'A large white shark was filmed divers a local wreck days prior the incident',
 'A long thin browncolored shark',
 'A pack sharks',
 'A shark known Old Tom',
 'A shark was seen the area witnesses',
 'A small shark',
 'A small spinner shark',
 'A yearold pound pregnant blacktip reef shark',
 'A \x93small\x94 shark',
 'According Benjamin the injury was inflicted a barracuda not shark',
 'According Carlsmith the sharks mouth was wide',
 'Alleged involve White shark m',
 'Alleged involve white shark with little yellow eyes',
 'Allegedly foot shark',
 'Allegedly tiger shark',
 'Angel shark',
 'Angel shark m',
 'Atkins remains were recovered from dusky shark obscurus Capt F Young shark fisherman',
 'Attacked a number sharks',
 'Basking shark',
 'Believed involve white shark',
 'Believed white shark and

In [13]:
df_sp = df_raw
df_sp.loc[(df_sp.Species == ''), 'Species'] = np.nan

df_sp.Species.fillna('Pepe shark', inplace = True)
df_sp.Species = df_sp.Species.str.lower()

df_sp1 = df_sp
df_sp1.Species = shark_sp(df_sp1.Species)
df_sp1.Species.value_counts()

# lst_sp = df_sp.Species.str.findall("\w+\sshark")
# lst_sp

# df_sp1 = df_sp
# df_sp1.Species = df_sp1.Species.str.findall("\w+\sshark")
# df_sp1.Species

# df_sp[df_sp.Species.str.contains("\w+\sshark")].Species.value_counts()


pepe shark                                                                              2944
shark                                                                                    699
white shark                                                                              299
white shark m                                                                            125
tiger shark                                                                              112
bull shark                                                                                93
shark involvement not confirmed                                                           82
tiger shark m                                                                             81
to m to shark                                                                             49
blacktip shark                                                                            34
wobbegong shark                                                       

In [14]:
# df_raw.Sex = df_raw.Sex.str.replace("\s$", "")
# print(df_Madre['Sex '].value_counts())

In [15]:
text = 'White sharkk asjdnajsdu'
# textCC = re.sub("(\w+\sshark)!", "", text)
# textCC

textCC = re.sub(r'(?!\w+\sshark)\b([\S\s]+?)(\b|$)', lambda x: (x.end() - x.start())*'', text)
textCC

'White'

In [16]:
# df_raw = df_Madre

# def fun(*args,**kwargs):
#     print("Generic FN:")
#     print("\tArgs", args)
#     print("\tKwargs", kwargs)
    
# def clean_last_ws (serie):
#     aux = re.sub("\s$", "", serie)
#     return aux
    
# new = df_raw[["Sex "]].values.tolist()
# new
# aux = re.sub("\s$", "", *new)
# aux